In [5]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from climate.helpers import *
from climate.climate import Weather
weather = Weather("./test/weatherfile.epw").read(psychrometrics=True, sun_position=True, sky_matrix=True)

Solar position calculations successful
Psychrometric calculations successful
WEA file created: /Users/Tristan/Desktop/climate/test/weatherfile.wea
Direct sky matrix calculated: /Users/Tristan/Desktop/climate/test/weatherfile.dirmtx
Diffuse sky matrix calculated: /Users/Tristan/Desktop/climate/test/weatherfile.diffmtx


# Define the annual method for calculation!!

In [12]:
class NV(object):

    def __init__(self, weather):
        
        # Hourly climate properties
        self.dry_bulb_temperature = weather.dry_bulb_temperature
        self.dew_point_temperature = weather.dew_point_temperature
        self.relative_humidity = weather.relative_humidity
        self.total_sky_cover = weather.total_sky_cover / 10
        self.sky_emissivity = None
        self.horizontal_infrared_radiation_intensity = weather.horizontal_infrared_radiation_intensity
        self.pedestrian_wind_speed = None
        
        # Ground/material properties
        self.ground_temperature = None
        self.ground_convective_heat_transfer_coefficient = None
        self.ground_k_value = None
        self.ground_emissivity = None
        self.ground_absorptivity = None
        self.ground_thickness = None
        self.ground_surface_temperature = None
        
        # Sky dome/radiation properties
        self.sky_dome_patch_radiation = None
        self.sky_dome_patch_centroids = None
        self.sample_vectors_issky = None
        self.sample_vectors_elevation = None
        self.sample_vectors = None
        
        
        
        # Methods
        def SkyEmissivity(self):
            return (0.787 + 0.764 * np.log((self.dew_point_temperature + 273.15) / 273.15)) * (1 + 0.0224 * self.total_sky_cover - 0.0035 * np.power(self.total_sky_cover, 2) + 0.0028 * np.power(self.total_sky_cover, 3))

# Load variables from elsewhere
nv = NV(weather)
nv.sky_emissivity



UnboundLocalError: local variable 'SkyEmissivity' referenced before assignment

# Development

In [3]:
nhour = 3183 - 1
weather.index[nhour]

Timestamp('2018-05-13 14:00:00+0000', tz='UTC', freq='60T')

In [27]:
# Pedestrian wind speed
pws = wind_speed_at_height(weather.wind_speed[nhour], h1=10, h2=2.0, rc=0.5, log=True)
# pws = 5.34834
pws

2.5013995959072965

In [28]:
gnd_temp = weather.ground_temperature_1[nhour]
dbt = weather.dry_bulb_temperature[nhour]
dpt = weather.dew_point_temperature[nhour]
rh = weather.relative_humidity[nhour]
tot_sky_cover = weather.total_sky_cover[nhour] / 10
hir = weather.horizontal_infrared_radiation_intensity[nhour]

(gnd_temp, dbt, dpt, rh, tot_sky_cover, hir)

(7.75, 11.0, 11.0, 100, 0.8, 332)

In [29]:
# Generate Numerous Vectors
weather.generate_numerous_vectors()
tsm = weather.total_sky_matrix[nhour]
vec = weather.nv_sample_vectors
alt = weather.nv_sample_thetas
sky = weather.nv_sample_sky

tsm[0:4], vec[0:4], alt[0:4], sky[0:4], 

(array([0.74866746, 0.74859408, 0.74859408, 0.7497263 ]),
 array([[ 0.04471018, -0.999     ,  0.        ],
        [-0.05707349, -0.997     ,  0.052284  ],
        [ 0.00873164, -0.995     , -0.0994925 ],
        [ 0.07186536, -0.993     ,  0.09373564]]),
 array([0.        , 0.05230785, 0.09965738, 0.09387345]),
 array([False,  True, False,  True]))

In [13]:
np.interp(np.power(weather.relative_humidity, 3), [0, 1e6], [0.33, 1.4])

array([1.19066199, 1.03459821, 1.03459821, ..., 1.03459821, 0.89864187,
       1.19066199])

In [30]:
# Surface Temperature "k"
k = np.interp(np.power(rh, 3), [0, 1e6], [0.33, 1.4])
k

1.4

In [31]:
weather.calculate_sky_emissivity()
se = weather.sky_emissivity[nhour]
se

0.8311482273571045

In [32]:
material = {'Concrete (Medium Rough)': {"D": 10.79, "E": 4.192, "F": 0}}
c_hc = material['Concrete (Medium Rough)']["D"] + material['Concrete (Medium Rough)']["E"] * pws + material['Concrete (Medium Rough)']["F"] * np.power(pws, 2)
c_hc

21.275867106043385

In [10]:
pc = weather.patch_centroids
pc[0:4]

array([[0.      , 0.997474, 0.054056],
       [0.104264, 0.99201 , 0.054056],
       [0.207387, 0.975677, 0.054056],
       [0.308237, 0.948654, 0.054056]])

In [11]:
weather.resample_sky_dome()
item = weather.nv_sample_indices
dis = weather.nv_sample_distances
item[0:3], dis[0:3]

(array([[30, 29, 31],
        [31, 30, 91],
        [30, 29, 31]]), array([[0.0701668 , 0.08073146, 0.15863233],
        [0.04748667, 0.05710296, 0.11968513],
        [0.15381647, 0.18086602, 0.19066751]]))

In [12]:
weather.calculate_sun_view_factor()
svf = weather.nv_sun_view_factor
svf[0:10]

array([0.        , 0.28872487, 0.        , 0.28932453, 0.        ,
       0.        , 0.28929047, 0.        , 0.28892727, 0.28910144])

In [13]:
weather.ray_trace_1000()
last_vector = weather.nv_last_ray_bounce_vector
len_int = weather.nv_intersected_points
last_vector[0:4], len_int[0:7], 

(array([[ 0.04471 , -0.999   ,  0.      ],
        [-0.057073, -0.997   ,  0.052284],
        [ 0.008732, -0.995   ,  0.099493],
        [ 0.071865, -0.993   ,  0.093736]]), array([0, 0, 1, 0, 1, 1, 0]))

In [14]:
# weather.closest_point(vec, last_vector[0])

from scipy import spatial
_, inds = spatial.KDTree(vec).query(last_vector, 1)
inds[0:7]

array([0, 1, 3, 3, 4, 8, 6])

In [15]:
weather.calculate_numerous_vector_radiation_values()
nvrad = weather.nv_radiation[nhour]
nvrad[0:8]
# Differences from interpretation of direct/diffuse radiaiton skly dome values - mine is better :)

array([0.        , 1.18559257, 0.        , 1.07918591, 0.        ,
       0.        , 1.14858183, 0.        ])

In [16]:
weather.m2_total_radiation()
m2rad = weather.nv_m2_radiation[nhour]
m2rad

458.08880400038174

In [17]:
weather.calculate_ein()
ein = weather.nv_ein[nhour]
ein[0:7]

array([0.        , 0.34231006, 0.31223495, 0.31223495, 0.        ,
       0.29723874, 0.33227377])

In [48]:
# Ein Out
ground_albedo = 0.2
Eout = sum(np.power(ground_albedo, len_int) * ein)
Eout

240.120035336094

In [58]:
emissivity = 0.8
absorptivity = 0.6
tickness = 0.8
# k = k
Ein = m2rad
Tin = gnd_temp
Ta = dbt
hc = c_hc

d=5.67*(10**(-8))
Tin=Tin+273.15
Ta=Ta+273.15
a=emissivity*d
b=k/tickness+hc
c=-(k*Tin/tickness+Ein*absorptivity+hc*Ta)


Ts=[]
nnn=[]
try:
    X = 1/2 *math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a))-1/2*math.sqrt(-(2*b)/(a *math.sqrt((4* (2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27 *a**2 *b**4-256* a**3* c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)))-(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)-(4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2* b**4-256* a**3*c**3)+9*a*b**2)**(1/3))
    if 0<X:
        Ts.append(X)
except: nnn.append(1)

try:
    X = 1/2 *math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a))+1/2*math.sqrt(-(2*b)/(a*math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)))-(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)-(4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3))
    if 0<X:
        Ts.append(X)
except: nnn.append(2)

try:
    X =-1/2 *math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a))-1/2*math.sqrt((2*b)/(a*math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)))-(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)-(4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3))
    if 0<X:
        Ts.append(X)
except: nnn.append(3)

try:
    X = 1/2 *math.sqrt((2*b)/(a*math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)))-(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a)-(4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3))-1/2*math.sqrt((4*(2/3)**(1/3)*c)/(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)+(math.sqrt(3)*math.sqrt(27*a**2*b**4-256*a**3*c**3)+9*a*b**2)**(1/3)/(2**(1/3)*3**(2/3)*a))
    if 0<X:
        Ts.append(X)
except: nnn.append(4)
print(nnn)

Tsc=[]
for i in Ts:
    Tsc.append(i-273.15)
e_out=[]
for i in Ts:
    E=a*(i**4)
    e_out.append(E)

# Surface tempertaure
Ts

[1, 2]


[283.1729880630722]

In [59]:
# Atmospheric Radiation

SVF = 1
TotalSky = se * hir * SVF

TotalSky

275.9412114825587

In [60]:
# Solar Radiation

σ= 5.6697*10**(-8)
ap=0.7
if Eout:
    TotalSol=ap*Eout
else: 
    TotalSol=0
TotalSol

168.08402473526579

In [63]:
#Ground radiation

#Eurb = σ.ε.Ts**4
#Floor Fv= 0.4

A1 = 1.2  # Human area
σ= 5.6697*10**(-8)
R=[]
A2=1
for t in Ts:
    if t:
        a=(A1/A2)  *   emissivity   *   (σ*emissivity*t**4)   *    .4
        R.append(a)
SR=sum(R)
SR

111.99240865983995

In [67]:
σ= 5.6697e-8
Tmrt = np.power(((TotalSky + TotalSol + SR) / σ), 0.25) - 273.15
Tmrt

41.53940663065487